<a href="https://colab.research.google.com/github/kocherlab/naps/blob/main/docs/notebooks/naps_basic_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook: Example NAPS usage

In this notebook we'll install NAPS, pull the example data from the [GitHub repository](https://github.com/kocherlab/naps), and run `naps-track` against it.

This repo is particularly useful in combination with SLEAP's example notebook on remote training and inference which can be found [here](https://colab.research.google.com/github/talmolab/sleap/blob/main/docs/notebooks/Training_and_inference_on_an_example_dataset.ipynb).

## Install NAPS

In [79]:
%%capture
%pip install -q naps-track

In [88]:
# If you have a model and want to do the inference on Colab, this can be done quite directly! Just upload your model and run inference as below.
# You can also take advantage of the GPU accessibility of Colab to train as well. Look to the SLEAP tutorials for more info.
# The models used for these videos can be found at https://doi.org/10.34770/6t6b-9545

# !sleap-track example.mp4 -o example-testing.slp -m naps_data/sleap-models/centroid -m naps_data/sleap-models/centered_instance --verbosity json --batch_size 1 --tracking.tracker simple --tracking.similarity iou --tracking.post_connect_single_breaks 1 --tracking.pre_cull_to_target 50 --tracking.target_instance_count 50 --tacking.clean_instance_count 50 --gpu 0

INFO:numexpr.utils:Note: detected 256 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 256 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
Started inference at: 2023-01-06 19:07:23.524253
Args:
{
│   'data_path': 'example.mp4',
│   'models': [
│   │   'naps_data/sleap-models/centroid',
│   │   'naps_data/sleap-models/centered_instance'
│   ],
│   'frames': '',
│   'only_labeled_frames': False,
│   'only_suggested_frames': False,
│   'output': 'example-testing.slp',
│   'no_empty_frames': False,
│   'verbosity': 'json',
│   'video.dataset': None,
│   'video.input_format': 'channels_last',
│   'video.index': '',
│   'cpu': False,
│   'first_gpu': False,
│   'last_gpu': False,
│   'gpu': 'auto',
│   'max_edge_length_ratio': 0.25,
│   'dist_penalty_weight': 1.0,
│   'batch_size': 1,
│   'open_in_gui': False,
│   'peak_threshol

## Download sample training data into Colab
Let's download a sample dataset from the the NAPS repository.


In [81]:
%%capture
!wget https://github.com/kocherlab/naps/raw/main/docs/notebooks/example_data/example.slp
!wget https://github.com/kocherlab/naps/raw/main/docs/notebooks/example_data/example.analysis.h5
!wget https://github.com/kocherlab/naps/raw/main/docs/notebooks/example_data/example.mp4

In [82]:
!ls -lht

total 331M
drwxr-sr-x. 2 swwolf julienlb  329 Jan  6 18:49 example_data
-rw-rw-r--. 1 swwolf julienlb 3.2M Jan  6 18:43 output.mp4
-rw-rw-r--. 1 swwolf julienlb 4.2M Jan  6 18:40 naps_basic_workflow.ipynb
-rw-rw-r--. 1 swwolf julienlb 7.9M Jan  6 17:47 example-naps.h5
-rw-rw-r--. 1 swwolf julienlb  29M Jan  6 17:47 example-naps.slp
-rw-rw-r--. 1 swwolf julienlb  86M Jan  6 17:40 naps_output.zip
-rw-r--r--. 1 swwolf julienlb  29M Jan  6 17:36 example.slp
-rw-rw-r--. 1 swwolf julienlb  56M Jan  6 17:36 example.mp4
-rw-rw-r--. 1 swwolf julienlb 8.0M Jan  6 17:36 example.analysis.h5
-rw-r--r--. 1 swwolf julienlb  51M Jan  6 17:36 example-aruco.npy
drwxr-sr-x. 8 swwolf julienlb  427 Jan  6 17:20 naps_data
drwxr-sr-x. 2 swwolf julienlb   30 Jan  6 16:30 utils
-rw-r--r--. 1 swwolf julienlb 219K Jan  6 15:54 naps_manuscript_graphics.ipynb
-rw-r--r--. 1 swwolf julienlb 532K Dec  7 13:27 SLEAP_metrics.ipynb
drwxr-sr-x. 2 swwolf julienlb  311 Dec  7 13:10 figures


## NAPS tracking
Now let's track the files using `naps-track`. We've adjusted a couple params here to make the tracks nicer.

In [1]:
!naps-track --slp-path example.slp --video-path example.mp4 --tag-node-name tag --start-frame 0 --end-frame 1200 --aruco-marker-set DICT_5X5_50 --output-path example-naps.slp --aruco-error-correction-rate 0.6 --aruco-adaptive-thresh-constant 7 --aruco-adaptive-thresh-win-size-max 23 --aruco-adaptive-thresh-win-size-step 10 --aruco-adaptive-thresh-win-size-min 3 --half-rolling-window-size 20

2023-01-06 19:33:12 INFO     Note: detected 256 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2023-01-06 19:33:12 INFO     Note: NumExpr detected 256 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-01-06 19:33:12 INFO     NumExpr defaulting to 8 threads.
2023-01-06 19:33:17 INFO     Loading predictions...
2023-01-06 19:33:24 INFO     Create ArUco model...
2023-01-06 19:33:24 INFO     ArUco model built in 1.5020370483398438e-05 seconds.
2023-01-06 19:33:24 INFO     Starting matching...
Processing frames 0 to 1200
2023-01-06 19:34:09 INFO     Done matching in 45.064929485321045 seconds.
2023-01-06 19:34:09 INFO     Reconstructing SLEAP file...
2023-01-06 19:34:15 INFO     Total tags: 45
2023-01-06 19:34:17 INFO     Done reconstructing SLEAP file in 7.422713756561279 seconds.
2023-01-06 19:34:17 INFO     Complete NAPS runtime: 59.63303208351135


## Download

Now we can just download the output! This pulls the video, the output file, and the original project. 

Once you have these, you can create a new SLEAP project and import example_output.analysis.h5 and point to the video to see the resulting tracks. If you are curious how these compare with the original tracks, you can open the original project.

In [ ]:
# Zip the video and output
!zip -0 -r naps_output.zip example.mp4 example.slp example-naps.slp

# Download
from google.colab import files
files.download("/content/naps_output.zip")

If you happen to not be using Chrome, you may get an error here. If that happens, you should be able to download the files using the "Files" tab on the left side panel.

## After NAPS

### SLEAP GUI

Now that we've got the files, we can either open the raw data or import it back into SLEAP. To view the tracks, open SLEAP (`sleap-label`) and create a new project. After creating a new project, you can go File > Import > SLEAP Analysis HDF5 and select the output, here example_output.analysis.h5. When you select the file, you will be prompted to select the video associated with the analysis file. You can simply select example.mp4 and then tracks will display in SLEAP.

In [ ]:
!sleap-convert example-naps.slp -o example-naps.h5 --format analysis

In [2]:
"""Read in the h5 and display basic info
"""

import h5py
import numpy as np

filename = "example-naps.h5"
video_filename = "example.mp4"
output_filename = "output.mp4"

with h5py.File(filename, "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

print("===filename===")
print(filename)
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

===filename===
example-naps.h5

===HDF5 datasets===
['edge_inds', 'edge_names', 'instance_scores', 'labels_path', 'node_names', 'point_scores', 'provenance', 'track_names', 'track_occupancy', 'tracking_scores', 'tracks', 'video_ind', 'video_path']

===locations data shape===
(1201, 17, 2, 45)

===nodes===
0: tag
1: head
2: thorax_a
3: abdomen
4: antenna_l
5: antenna_r
6: foreleg_l
7: foreleg_r
8: midleg_l
9: midleg_r
10: hindleg_l
11: hindleg_r
12: wing_l
13: wing_r
14: thorax_h
15: antenna_l_joint
16: antenna_r_joint



## Rendering with sleap-render
Now we can simply render this as a video using `sleap-render`. We'll use --frames to subset the frames to render and --tracks to render the tracks. We'll also use --output to specify the output file name.

In [3]:
!sleap-render example-naps.slp --frames 550-650 --crop 1024,1024 -o example-naps-tracks.mp4

INFO:numexpr.utils:Note: detected 256 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 256 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
Saving config: /Genomics/argo/users/swwolf/.sleap/1.2.8/preferences.yaml
Writing video with 201 frame images...
INFO:sleap.io.visuals:Chunks: 4, chunk size: 64
Finished 64 frames in 72.4 s, fps = 1, approx 155.0 s remaining
Finished 128 frames in 128.3 s, fps = 1, approx 73.2 s remaining


In [ ]:
show_video("example-naps-tracks.mp4")